## Experiment: Regelkonformität von Transformer-Modellen anhand Wikipedia-N-Grammen

In diesem Experiment wird untersucht, wie gut zwei verschiedene Transformer-Modelle durch modellinterne N-Gramm-Regeln beschrieben werden können. Hierzu werden beide Modelle auf dieselben Wikipedia-Sätze angewendet, aber jeweils mit eigens berechneten N-Gramm-Regeln, die auf dem Tokenisierungsschema des jeweiligen Modells basieren.

Ziel ist es, zu überprüfen, ob ein Modell wie TinyLlama 1.1B, trotz generischem Training, ähnliche regelhafte Strukturen aufweist wie ein auf einfache Texte spezialisiertes Modell (TinyStories-Modell).

Die Vergleichbarkeit erfolgt dabei nicht über gemeinsame Regeln, sondern über die Frage, wie stark sich jedes Modell mit seiner eigenen Regelbasis beschreiben lässt.

## 1. Hintergrund

Im Originalpaper von **Nguyen(2024)** wird gezeigt, dass Transformer-Modelle bis zu einem gewissen Grad durch einfache statistische Regeln – sogenannte N-Gramm-Regeln – beschrieben werden können. Diese Regeln basieren auf Häufigkeiten von Tokenfolgen in den Trainingsdaten. Im Paper wurden TinyStories-Regeln genutzt, um Modellverhalten zu beschreiben. In diesem Experiment werden stattdessen eigene Regeln aus Wikipedia-Sätzen berechnet, um einen fairen Modellvergleich zu ermöglichen.

## 2. Ziel des Experiments

In diesem zweiten Experiment wird geprüft, wie gut TinyLlama 1.1B im Vergleich zum TinyStories-Modell durch eigene N-Gramm-Regeln aus Wikipedia beschrieben werden kann.



## 3. Vorgehensweise

### Modelle:
- **Eigenes Modell**: Trainiert auf TinyStories.
- **TinyLlama 1.1B**: Allgemeines Decoder-Only-Modell von Google, SentencePiece-Tokenizer.

### Daten:
- **Wikipedia**: zufällig Ausgewählte Sätze aus 100-500 Artikeln aus dem englischen Wikipedia-Datensatz (2022-03-01). Es werden nur Sätze mit mindestens 6 Wörtern berücksichtigt

### Schritte:
1. Wikipedia-Sätze tokenisieren (jeweils pro Modell mit eigenem Tokenizer).
2. N-Gramm-Statistiken berechnen → pro Modell eigene Regeln erstellen.
3. Für jeden Satz: Modellvorhersagen berechnen (eigenes Modell & TinyLlama).
4. Pro Vorhersage: Beste passende Regel aus eigenen N-Grammen finden.
5. Vergleich pro Modell:
- Top-1 / Top-3 Accuracy: Stimmen Regel- und Modellvorhersage überein?
- Variationsdistanz: Wie ähnlich sind die Wahrscheinlichkeitsverteilungen?
## 4. Hypothese

Da TinyLlama 1.1B auf vielfältigen Daten trainiert wurde, könnte es weniger regelhaft agieren als ein auf TinyStories spezialisiertes Modell. Dies soll durch die Regelapproximation anhand eigener Wikipedia-Regeln überprüft werden. Zusätzlich wird untersucht, ob Modelle bei häufigeren Kontexten (z. B. mit ≥ 3 Vorkommen) besser durch Regeln beschrieben werden können, wie es auch im Originalpaper gezeigt wurde.


In [2]:
import torch
import sentencepiece as spm
import gcsfs
import time
from transformers import AutoModelForCausalLM, AutoTokenizer
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Gerät festlegen 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenizer-Pfad
TOKENIZER_PATH = "gs://transformer-ngrams/32768.model"
LOCAL_TOKENIZER_PATH = "32768.model"
VOCAB_SIZE = 32768
BOS_TOKEN = 1

def download_tokenizer_from_gcs(gcs_path, local_path):
    print(f"Lade Tokenizer von GCS: {gcs_path} ...")
    fs = gcsfs.GCSFileSystem()
    with fs.open(gcs_path, 'rb') as gcs_file, open(local_path, 'wb') as local_file:
        local_file.write(gcs_file.read())
    print(f"Tokenizer erfolgreich heruntergeladen nach: {local_path}")

def load_author_tokenizer(tokenizer_path=TOKENIZER_PATH):
    print(f"Lade SentencePiece-Tokenizer von: {tokenizer_path}")
    tokenizer = spm.SentencePieceProcessor()
    tokenizer.load(tokenizer_path)
    print("Autorentokenizer erfolgreich geladen!")
    return tokenizer

# Tokenizer laden
download_tokenizer_from_gcs(TOKENIZER_PATH, LOCAL_TOKENIZER_PATH)
tokenizer = load_author_tokenizer(LOCAL_TOKENIZER_PATH)

# Überprüfung 
print(tokenizer.decode_ids([32599, 32600, 9, 375, 586]))

# Eigenes Modell laden 
MODEL_NAME = "dadosbon/TSModel2Try_continuation_1epoch"
def load_transformer_model(model_name=MODEL_NAME):
    print(f"Lade Transformer-Modell: {model_name} ...")
    model = AutoModelForCausalLM.from_pretrained(model_name)
    print("Modell erfolgreich geladen!")
    return model

# Zeitmessung starten 
start_time = time.time()

# Modell 1 laden 
model1 = load_transformer_model().to(device)

#  Modell 2 (TinyLlama) laden 
model2 = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0").to(device)
tokenizer2 = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Bestätigen, dass beide auf GPU sind 
print(f"Modell 1 auf: {next(model1.parameters()).device}")
print(f"Modell 2 auf: {next(model2.parameters()).device}")


Lade Tokenizer von GCS: gs://transformer-ngrams/32768.model ...
Tokenizer erfolgreich heruntergeladen nach: 32768.model
Lade SentencePiece-Tokenizer von: 32768.model
Autorentokenizer erfolgreich geladen!
.,1 (ll
Lade Transformer-Modell: dadosbon/TSModel2Try_continuation_1epoch ...
Modell erfolgreich geladen!
Modell 1 auf: cuda:0
Modell 2 auf: cuda:0


# Datenvorbereitung: Wikipedia-Sätze extrahieren
In diesem Abschnitt wird eine **zufällige Auswahl** an Sätzen aus dem englischen Wikipedia-Datensatz getroffen. Diese Sätze dienen später zur Regelapproximation durch Transformer-Modelle.

**Schritte**:
1. Laden des Wikipedia-Datensatzes (Stand: 2022-03-01)

2. Auswahl von 400 zufälligen Artikeln

3. Extraktion von mindestens 6 Wörter langen Sätzen

4. Zufällige Auswahl von 2000 Sätzen für das Experiment

5. Kontrolle der Tokenisierung mit dem TinyStories-Tokenizer

In [3]:
from datasets import load_dataset
import random
import re

# 1. Wikipedia-Datensatz laden (Version 20220301)
wiki_dataset = load_dataset("wikipedia", "20220301.en", split="train", trust_remote_code=True)

# 2. 100 zufällige Artikel auswählen
random.seed(42)
sampled_articles = random.sample(list(wiki_dataset), 400)

# 3. Nur Text extrahieren
wiki_texts = [entry["text"] for entry in sampled_articles]

# 4. Sätze extrahieren (min. 6 Wörter)
def extract_sentences(text):
    sentences = re.split(r'[.!?]\s+', text)
    return [s.strip() for s in sentences if len(s.strip().split()) > 5]

# 5. Alle Sätze extrahieren
wiki_sentences = []
for text in wiki_texts:
    wiki_sentences.extend(extract_sentences(text))

# 6. 3000 zufällige Sätze auswählen
wiki_sentences_sampled = random.sample(wiki_sentences, 2000)

# 7. Beispiel-Tokenisierung (TinyStories-Tokenizer) – Kontrolle
for i, sentence in enumerate(wiki_sentences_sampled[:5]):
    tokens = tokenizer.encode_as_pieces(sentence)
    print(f"{i+1}: {sentence}")
    print(f"Tokens: {tokens}\n")


Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

1: In 1863, the Virginia Peninsula community gathered under this tree to hear the first Southern reading of President Abraham Lincoln's Emancipation Proclamation, and it became known as the Emancipation Oak
Tokens: ['In', '▁', '1', '8', '6', '3', ',', '▁the', '▁Virginia', '▁Peninsula', '▁community', '▁gathered', '▁under', '▁this', '▁tree', '▁to', '▁hear', '▁the', '▁first', '▁Southern', '▁reading', '▁of', '▁President', '▁Abraham', '▁Lincoln', "'", 's', '▁E', 'man', 'c', 'ipation', '▁Pro', 'clamation', ',', '▁and', '▁it', '▁became', '▁known', '▁as', '▁the', '▁E', 'man', 'c', 'ipation', '▁Oak']

2: 99.44% were from white (including Irish Traveller) ethnic groups;
 62.41% belong to or were brought up in the Catholic religion and 34.77% belong to or were brought up in a 'Protestant and Other Christian (including Christian related)' religion; and
 30.18% indicated that they had a British national identity, 44.39% had an Irish national identity and 27.82% had a Northern Irish national identit

In [5]:
import collections
import pandas as pd
from typing import List, Tuple


# Google Cloud Storage Einstellungen
GCS_BUCKET = 'gs://transformer-ngrams/TinyStories/train_data_rules'
fs = gcsfs.GCSFileSystem()


# Berechnung der N-Gramm-Regeln aus Wikipedia-Sätzen

In diesem Schritt werden N-Gramm-Regeln auf Basis der Wikipedia-Daten extrahiert. Dazu werden alle Sätze für beide Modelle separat tokenisiert. Anschließend werden 7-Gramm-Kontexte mit dem jeweils folgenden Token gezählt.

Für beide Modelle wird daraus ein N-Gramm-Zähler aufgebaut, der für jeden Kontext eine Häufigkeitsverteilung möglicher nächster Tokens enthält. Diese Verteilung dient als Regel-Approximation im späteren Vergleich.

Zusätzlich wird jeweils ein Beispielkontext ausgegeben, zusammen mit der zugehörigen Regelverteilung. So lässt sich direkt prüfen, ob sinnvolle Regeln entstanden sind.

**Bei TinyLlama wird `add_special_tokens=False` verwendet, um BOS-/EOS-Tokens zu vermeiden und eine konsistente Tokenisierung zu sichern**


In [6]:
from collections import defaultdict

# Berechnet N-Gramm Häufigkeiten
def count_ngrams(tokenized_sentences: list, N: int = 7):
    ngram_counts = defaultdict(lambda: defaultdict(int))
    for tokens in tokenized_sentences:
        for i in range(len(tokens) - N):
            context = tuple(tokens[i:i+N])
            next_token = tokens[i+N]
            ngram_counts[context][next_token] += 1
    return ngram_counts


# ------------------------Eigenes Modell----------------------------------

# Wikipedia-Sätze tokenisieren 
tokenized_sentences_own = [tokenizer.encode_as_ids(sentence) for sentence in wiki_sentences_sampled]

# N-Gramme zählen
ngram_counts_own = count_ngrams(tokenized_sentences_own, N=7)
print(f"Anzahl eindeutiger Kontexte: {len(ngram_counts_own)}")

# Regelverteilung berechnen
def compute_rule_probabilities(context, ngram_counts):
    next_token_counts = ngram_counts.get(context, {})
    total = sum(next_token_counts.values())
    if total == 0:
        return None
    return {token: count / total for token, count in next_token_counts.items()}

# Beispiel-Regel aus Kontext berechnen
sample_context = next(iter(ngram_counts_own.keys()))
rule_probs_own = compute_rule_probabilities(sample_context, ngram_counts_own)

print("Kontext (Token-IDs):", sample_context)
print("Regelverteilung:", list(rule_probs_own.items())[:5])

#--------------------------Modell2 TinyLlama----------------------------

# Wikipedia-Sätze tokenisieren (TinyLlama Tokenizer)
tokenized_sentences_tinyllama = [tokenizer2(sentence, return_tensors="pt", add_special_tokens=False)["input_ids"][0].tolist()
                                 for sentence in wiki_sentences_sampled]

#N-Gramme zählen
ngram_counts_tinyllama = count_ngrams(tokenized_sentences_tinyllama, N=7)
print(f"Anzahl eindeutiger Kontexte (TinyLlama): {len(ngram_counts_tinyllama)}")

#Beispielregel berechnen
sample_context_tinyllama = next(iter(ngram_counts_tinyllama.keys()))
rule_probs_tinyllama = compute_rule_probabilities(sample_context_tinyllama, ngram_counts_tinyllama)

print("TinyLlama-Kontext (Token-IDs):", sample_context_tinyllama)
print("TinyLlama-Regelverteilung:", list(rule_probs_tinyllama.items())[:5])




Anzahl eindeutiger Kontexte: 65672
Kontext (Token-IDs): (579, 32578, 9, 16, 14, 11, 32600)
Regelverteilung: [(280, 1.0)]
Anzahl eindeutiger Kontexte (TinyLlama): 70637
TinyLlama-Kontext (Token-IDs): (512, 29871, 29896, 29947, 29953, 29941, 29892)
TinyLlama-Regelverteilung: [(278, 1.0)]


# Regelmatching Suffix

Diese Funktion implementiert das **Suffix-Matching**:
Anstatt nur exakte Kontextlängen zuzulassen, wird schrittweise geprüft, ob ein verkürzter Kontext (Suffix) im N-Gramm-Zähler enthalten ist. Dies erhöht die Chance, überhaupt eine Regelverteilung zu finden, insbesondere bei seltenen oder neuen Konstellationen.

Das Matching beginnt mit dem vollen Kontext (z. B. 7 Token) und reduziert sich iterativ bis auf Länge 1. Sobald ein gültiger Kontext im Zähler gefunden wird, wird die zugehörige Regelverteilung zurückgegeben.


In [7]:
def find_matching_rule(context_tokens, ngram_counts, N=2):
    """
    Verwendet die N-Gramm-Zähler für den nächsten möglichen Token (N-Gramm Regel).
    Stellt sicher, dass mehr Tokens verglichen werden, falls der Kontext wenig Treffer liefert.
    """
    context_length = len(context_tokens)
    for match_len in range(context_length, 0, -1):
        suffix = tuple(context_tokens[-match_len:])
        if suffix in ngram_counts:
            # Rückgabe mehrerer möglicher Tokens
            return compute_rule_probabilities(suffix, ngram_counts)
    return None

# Modell-Regel Vergleich

Für jeden Satz wird die letzte Vorhersage des Modells (Token t<sub>i</sub>) mit der durch N-Gramme abgeleiteten Regelverteilung für den vorherigen Kontext verglichen.

Drei Metriken werden berechnet:
- **Top‑1 Accuracy**: Stimmt das wahrscheinlichste Modell-Token mit der Regel überein?
- **Variationsdistanz**: Wie ähnlich sind Modellverteilung und Regelverteilung?
- **Top-3 Accuracy**: Ist der wahrscheinlichste Regel-Token unter den Top‑3-Vorhersagen des Modells enthalten?

Zusätzlich wird ein Mindestvorkommen (`count ≥ 3`) gefiltert, um nur stabile Regeln zu vergleichen.


In [8]:
import numpy as np
import torch
import time

# Vergleich von Modellvorhersage und Regelverteilung
def compare_model_rule(model_probs: np.ndarray, rule_probs: dict, topk: int=1):
    if rule_probs is None:
        return 0, 1.0
    # Top-1 aus Modell
    model_top1 = np.argmax(model_probs)
    rule_top1 = max(rule_probs.items(), key=lambda x: x[1])[0]
    top1_acc = int(model_top1 == rule_top1)

     # Top‑k aus Modell
    model_topk = np.argpartition(-model_probs, topk)[:topk]
    topk_acc = int(rule_top1 in model_topk)

    # Regel als Vektor
    rule_vec = np.zeros_like(model_probs)
    for token_id, prob in rule_probs.items():
        if token_id < len(model_probs):
            rule_vec[token_id] = prob

    # Variationsdistanz
    variational_distance = 0.5 * np.sum(np.abs(model_probs - rule_vec))
    return top1_acc, variational_distance, topk_acc


# Ergebnislisten initialisieren
top1_accuracy_own = []
distances_own = []

top1_accuracy_tinyllama = []
distances_tinyllama = []

# Zeitmessung starten
start_total = time.time()
own_total_time = 0.0
tinyllama_total_time = 0.0

for idx, sentence in enumerate(wiki_sentences_sampled):
    print(f"Satz {idx+1}/100")

# -------------------------------------- Eigenes Modell -----------------------------------------------------
    token_ids = tokenizer.encode_as_ids(sentence)
    if len(token_ids) < 8:
        print("Satz zu kurz für Kontext – übersprungen.")
        continue
    
    context = token_ids[-8:-1]
    next_token = token_ids[-1]

    count = sum(ngram_counts_own.get(tuple(context), {}).values())
    if count < 3:
        print(f"[Eigenes Modell] Kontext zu selten – übersprungen (count = {count})")
        continue


    start_own = time.time()
    input_ids_own = torch.tensor([token_ids]).to(model1.device)
    with torch.no_grad():
        logits_own = model1(input_ids_own).logits
        probs_own = torch.nn.functional.softmax(logits_own[0, -1], dim=-1).cpu().numpy()
    end_own = time.time()
    own_total_time += (end_own - start_own)

    rule_probs_own = find_matching_rule(context, ngram_counts_own)

    if rule_probs_own is None:
        print(f"[Eigenes Modell] Keine passende Regel – Satz {idx+1}")
        continue 
                                    
    top1_acc_own, dist_own, top3_acc_own = compare_model_rule(probs_own, rule_probs_own, topk=3)
    top1_accuracy_own.append(top1_acc_own)
    distances_own.append(dist_own)
    print(f"[Eigenes Modell] Modell Top-1: {np.argmax(probs_own)}, Regel Top-1: {max(rule_probs_own.items(), key=lambda x: x[1])[0]}")
    print(f"Distanz: {dist_own:.3f}")


#--------------------------------------- TinyLlama --------------------------------------------------------------
    inputs_tinyllama = tokenizer2(sentence, return_tensors="pt", add_special_tokens=False)
    inputs_tinyllama = {k: v.to(model2.device) for k, v in inputs_tinyllama.items()}

    tokens_tinyllama = inputs_tinyllama["input_ids"][0].tolist()
    if len(tokens_tinyllama) < 8:
        print("TinyLlama-Satz zu kurz – übersprungen.")
        continue

    context_tinyllama = tokens_tinyllama[-8:-1]
    next_token_tinyllama = tokens_tinyllama[-1]

    count = sum(ngram_counts_tinyllama.get(tuple(context_tinyllama), {}).values())
    if count < 3:
        print(f"[TinyLlama] Kontext zu selten – übersprungen (count = {count})")
        continue


    rule_probs_tinyllama = find_matching_rule(context_tinyllama, ngram_counts_tinyllama)

    if rule_probs_tinyllama is None:
        print(f"[TinyLlama] Keine passende Regel – Satz {idx+1}")
        continue

    start_tinyllama = time.time()
    with torch.no_grad():
        logits_tinyllama = model2(**inputs_tinyllama).logits
        probs_tinyllama = torch.nn.functional.softmax(logits_tinyllama[0, -1], dim=-1).cpu().numpy()
    end_tinyllama = time.time()
    tinyllama_total_time += (end_tinyllama - start_tinyllama)

    top1_acc_tinyllama, dist_tinyllama, top3_acc_tinyllama = compare_model_rule(probs_tinyllama, rule_probs_tinyllama,topk=3)
    top1_accuracy_tinyllama.append(top1_acc_tinyllama)
    distances_tinyllama.append(dist_tinyllama)
    print(f"[TinyLlama] Modell Top-1: {np.argmax(probs_tinyllama)}, Regel Top-1: {max(rule_probs_tinyllama.items(), key=lambda x: x[1])[0]}")
    print(f"Distanz: {dist_tinyllama:.3f}")

processed_own = len(top1_accuracy_own)
# Zeitmessung beenden
end_total = time.time()
total_duration = end_total - start_total

# Durchschnittszeiten
num_vergleiche = len(top1_accuracy_own)
avg_own_time = own_total_time / num_vergleiche if num_vergleiche else 0
avg_tinyllama_time = tinyllama_total_time / num_vergleiche if num_vergleiche else 0

print("\n--- Zeitmessung ---")
print(f"Gesamtzeit: {total_duration:.2f} Sekunden")
print(f"Durchschnittliche Eigenes Modell-Zeit/Vorhersage: {avg_own_time:.3f} s")
print(f"Durchschnittliche TinyLlama-Zeit/Vorhersage: {avg_tinyllama_time:.3f} s")
print(f"Vergleiche durchgeführt: {num_vergleiche}")

# Mittelwerte & Standardabweichung
means_acc = [np.mean(top1_accuracy_tinyllama), np.mean(top1_accuracy_own)]
stds_acc = [np.std(top1_accuracy_tinyllama), np.std(top1_accuracy_own)]

means_dist = [np.mean(distances_tinyllama), np.mean(distances_own)]
stds_dist = [np.std(distances_tinyllama), np.std(distances_own)]

print(f"TinyLlama – Accuracy: {means_acc[0]:.3f} ± {stds_acc[0]:.3f}, Distanz: {means_dist[0]:.3f} ± {stds_dist[0]:.3f}")
print(f"Eigenes Modell – Accuracy: {means_acc[1]:.3f} ± {stds_acc[1]:.3f}, Distanz: {means_dist[1]:.3f} ± {stds_dist[1]:.3f}")
print(f"Sätze mit Vergleich Eigenes Modell: {processed_own}")
print("Kontext (Eigenes Modell):", context)
print("Beispiel-Kontext aus N-Gramm-Zähler:", next(iter(ngram_counts_own.keys())))


Satz 1/100
[Eigenes Modell] Kontext zu selten – übersprungen (count = 1)
Satz 2/100
[Eigenes Modell] Kontext zu selten – übersprungen (count = 1)
Satz 3/100
[Eigenes Modell] Kontext zu selten – übersprungen (count = 1)
Satz 4/100
[Eigenes Modell] Kontext zu selten – übersprungen (count = 1)
Satz 5/100
[Eigenes Modell] Kontext zu selten – übersprungen (count = 1)
Satz 6/100
[Eigenes Modell] Kontext zu selten – übersprungen (count = 1)
Satz 7/100
[Eigenes Modell] Kontext zu selten – übersprungen (count = 1)
Satz 8/100
[Eigenes Modell] Kontext zu selten – übersprungen (count = 1)
Satz 9/100
[Eigenes Modell] Kontext zu selten – übersprungen (count = 1)
Satz 10/100
[Eigenes Modell] Kontext zu selten – übersprungen (count = 1)
Satz 11/100
[Eigenes Modell] Kontext zu selten – übersprungen (count = 2)
Satz 12/100
[Eigenes Modell] Kontext zu selten – übersprungen (count = 1)
Satz 13/100
[Eigenes Modell] Kontext zu selten – übersprungen (count = 1)
Satz 14/100
[Eigenes Modell] Kontext zu selten 

In [9]:
# Ergebnislisten für Top-3 Accuracy
top3_accuracy_own = []
top3_accuracy_tinyllama = []

# Berechnungen für das eigene Modell (Top-1 und Top-3)
top1_acc_own, _, dist_own = compare_model_rule(probs_own, rule_probs_own, topk=1)
_, _, top3_acc_own = compare_model_rule(probs_own, rule_probs_own, topk=3)

top1_accuracy_own.append(top1_acc_own)
top3_accuracy_own.append(top3_acc_own)
distances_own.append(dist_own)

# Berechnungen für TinyLlama (Top-1 und Top-3)
top1_acc_tinyllama, _, dist_tinyllama = compare_model_rule(probs_tinyllama, rule_probs_tinyllama, topk=1)
_, _, top3_acc_tinyllama = compare_model_rule(probs_tinyllama, rule_probs_tinyllama, topk=3)

top1_accuracy_tinyllama.append(top1_acc_tinyllama)
top3_accuracy_tinyllama.append(top3_acc_tinyllama)
distances_tinyllama.append(dist_tinyllama)

# Mittelwerte & Standardabweichungen für Accuracy
means_top1 = [np.mean(top1_accuracy_tinyllama), np.mean(top1_accuracy_own)]
stds_top1 = [np.std(top1_accuracy_tinyllama), np.std(top1_accuracy_own)]

means_top3 = [np.mean(top3_accuracy_tinyllama), np.mean(top3_accuracy_own)]
stds_top3 = [np.std(top3_accuracy_tinyllama), np.std(top3_accuracy_own)]

# Ergebnisübersicht ausgeben
print("\n--- Ergebnisübersicht ---")
print(f"TinyLlama:")
print(f"  Top-1 Accuracy  : {means_top1[0]:.3f} ± {stds_top1[0]:.3f}")
print(f"  Top-3 Accuracy  : {means_top3[0]:.3f} ± {stds_top3[0]:.3f}")
print(f"  Distanz         : {means_dist[0]:.3f} ± {stds_dist[0]:.3f}")

print(f"Eigenes Modell:")
print(f"  Top-1 Accuracy  : {means_top1[1]:.3f} ± {stds_top1[1]:.3f}")
print(f"  Top-3 Accuracy  : {means_top3[1]:.3f} ± {stds_top3[1]:.3f}")
print(f"  Distanz         : {means_dist[1]:.3f} ± {stds_dist[1]:.3f}")



--- Ergebnisübersicht ---
TinyLlama:
  Top-1 Accuracy  : 0.000 ± 0.000
  Top-3 Accuracy  : 0.000 ± 0.000
  Distanz         : 1.000 ± 0.000
Eigenes Modell:
  Top-1 Accuracy  : 0.000 ± 0.000
  Top-3 Accuracy  : 0.000 ± 0.000
  Distanz         : 0.999 ± 0.003


# Zusammenfasung:

In diesem Experiment wurde untersucht, inwieweit zwei unterschiedliche Transformer-Modelle, TinyLlama 1.1B und ein eigens auf TinyStories trainiertes Modell, durch N-Gramm-Regeln approximiert werden können, die aus Wikipedia extrahiert wurden. Ziel war es zu analysieren, wie gut sich solche Regeln auf Modelle übertragen lassen, die auf anderen Datensätzen trainiert wurden.

Für jedes Modell wurde jeweils die passende N-Gramm-Regel gesucht, deren Wahrscheinlichkeitsverteilung die geringste Variationsdistanz zur tatsächlichen Modellverteilung aufwies. Dieser Ansatz entspricht der Methodik von **Nguyen (2024)**, wobei statt TinyStories-Regeln hier eigene Wikipedia-basierte Regeln verwendet wurden.

Dadurch konnte systematisch geprüft werden, wie gut sich einfache statistische Regeln, auch außerhalb ihrer Ursprungsdomäne, zur Beschreibung von Modellverhalten eignen.
# Ergebnisse:
TinyLlama 1.1B:

Top-1 Accuracy: 0.000 ± 0.000

Top-3 Accuracy: 0.000 ± 0.000

Distanz: 1.000 ± 0.000

---------------------------

Eigenes Modell (TinyStories):

Top-1 Accuracy: 0.000 ± 0.000

Top-3 Accuracy: 0.000 ± 0.000

Distanz: 0.999 ± 0.003

------------------------------

# Interpretation
Beide Modelle erzielen eine Top-1- und Top-3-Accuracy von 0 %, d.h. keine einzige Modellvorhersage stimmte mit der aus den N-Gramm-Regeln abgeleiteten Vorhersage überein. Das war zu erwarten: Selbst im Originalpaper erreichte der beste Regelsatz auf dem Trainingsdatensatz nur rund 78 % Top-1 Accuracy (vgl. Tabelle 2).

Besonders auffällig ist jedoch die hohe Variationsdistanz bei beiden Modellen:

- TinyLlama: 1.000 ± 0.000

- Eigenes Modell: 0.999 ± 0.003

Diese Werte deuten darauf hin, dass keiner der Modelloutputs der durch Regeln berechneten Verteilung auch nur **annähernd** ähnelt. Selbst das auf TinyStories trainierte Modell zeigt bei Wikipedia-Kontexten keine nennenswerte Übereinstimmung mit den Regelwahrscheinlichkeiten.

Im Gegensatz zum Versuch im Originalpaper, wo Regelverteilungen oft zumindest eine grobe Annäherung an die Modellverteilungen ermöglichten, schlagen die Regeln hier vollständig fehl. Das legt nahe, dass:

* die Modelle nicht regelkonform im Sinne der N-Gramm-Statistik handeln (zumindest nicht auf Wikipedia),

* oder dass die N-Gramm-Regeln aus Wikipedia schlicht keine gute Beschreibung für Modellverhalten liefern, dass auf anderen Datensätzen wie TinyStories basiert.

Diese hohe Distanz ist damit ein starker Hinweis auf **domänenspezifisches Verhalten**: Die Regelverteilungen aus Wikipedia erfassen offensichtlich nicht die Wahrscheinlichkeitsstruktur der Modelle – weder bei einem großen Modell wie TinyLlama noch beim spezialisierten TinyStories-Modell.

# Fazit 
Die Ergebnisse zeigen deutlich, dass N-Gramm-Regeln moderne Sprachmodelle – zumindest in dieser einfachen Form und domänenübergreifend – nicht zuverlässig approximieren können. Sowohl das auf TinyStories spezialisierte Modell als auch das allgemeine Sprachmodell TinyLlama 1.1B konnten keine einzige **Top-1- oder Top-3-Vorhersage** durch eine Wikipedia-basierte N-Gramm-Regel erklären.

Auch die Variationsdistanzen lagen bei beiden Modellen nahe **1**, was bedeutet, dass die Wahrscheinlichkeitsverteilungen der Modelle stark von den durch N-Gramme erzeugten Regeln abweichen. Das gilt sogar für das TinyStories-Modell, obwohl es auf einfache und strukturierte Texte trainiert wurde. Für komplexere, realweltliche Texte wie Wikipedia scheint seine Regelkonformität komplett zu versagen.

Diese Befunde stützen die zentrale Aussage des Originalpapers: *Selbst mit sorgfältig konstruierten Regeln und gut abgestimmten Kontextlängen bleibt die Regelapproximation für neuronale Sprachmodelle eine grobe Vereinfachung, die in der Praxis nur begrenzt übertragbar ist, insbesondere nicht zwischen unterschiedlichen Domänen*.